# Создаём точку входа в Spark

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
!pip install pyspark

In [56]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [57]:
spark

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

# Считывание данных

In [58]:
# Прочитайте содержимое каждого файла в датафрейм и выведите первые несколько строк (1 балл)
# Применив cast к колонке можно изменить тип данных (F.col().cast(IntegerType()).alias())
# Приведите все данные к правильному типу, либо считайе сразу верно (1 балл)
###############
# Ваш код здесь
data_test = spark.read.option('header','True').option('inferSchema', 'true').csv('/content/drive/MyDrive/Colab Notebooks/Spark/test.csv',sep=',')
data_test.show(5)

+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
| id|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|  1|         1043|   1|        1.8|       1| 14|     0|         5|  0.1|      193|      3| 16|      226|    1412|3476|  12|   7|        2|      0|           1|   0|
|  2|          841|   1|        0.5|       1|  4|     1|        61|  0.8|      191|      5| 12|      746|     857|3895|   6|   0|        7|      1|           0|   0|
|  3|         1807|   1|        2.8|       0|  1|     0|        27|  0.9|      186|      3|  4|     1270|    1366|2396|  17|  10|       10|      0|           1|   1|
|  4

In [59]:
#итак мы видим, что все данные испортированы с корректным типом данных
data_test.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, battery_power: int, blue: int, clock_speed: double, dual_sim: int, fc: int, four_g: int, int_memory: int, m_dep: double, mobile_wt: int, n_cores: int, pc: int, px_height: int, px_width: int, ram: int, sc_h: int, sc_w: int, talk_time: int, three_g: int, touch_screen: int, wifi: int]>

In [60]:
#аналогично импортируем датасет трэйн
data_train = spark.read.option('header','True').option('inferSchema', 'true').csv('/content/drive/MyDrive/Colab Notebooks/Spark/train.csv',sep=',')
data_train.show(5)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [61]:
data_train.printSchema

<bound method DataFrame.printSchema of DataFrame[battery_power: int, blue: int, clock_speed: double, dual_sim: int, fc: int, four_g: int, int_memory: int, m_dep: double, mobile_wt: int, n_cores: int, pc: int, px_height: int, px_width: int, ram: int, sc_h: int, sc_w: int, talk_time: int, three_g: int, touch_screen: int, wifi: int, price_range: int]>

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html)

PS: нужно сделать средствами pyspark pd.concat([train, test,], axis=0)

In [46]:
# Объедините train и test (1 балла)
###############
# Ваш код здесь

# чтоб объединить две таблицы, нужно обоспечить одинковое количество колонок.
# на прошлом уроке 
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType, IntegerType

data = data_test.withColumn('price_range', lit(None).cast(StringType())).unionByName(data_train.withColumn('id',lit(None).cast(StringType())))
data.show()

+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
| id|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|  1|         1043|   1|        1.8|       1| 14|     0|         5|  0.1|      193|      3| 16|      226|    1412|3476|  12|   7|        2|      0|           1|   0|       null|
|  2|          841|   1|        0.5|       1|  4|     1|        61|  0.8|      191|      5| 12|      746|     857|3895|   6|   0|        7|      1|           0|   0|       null|
|  3|         1807|   1|        2.8|       0|  1|     0|        27|  0.9|      186|      3|  4|     1270|    1

## Рассчитайте соотношение сторон телефона и экрана

In [78]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# в пикселях на дюйм
###############
# Ваш код здесь

#Добавим колонку диагональ в дюймах
from pyspark.sql.functions import sqrt, desc

#data.withColumn("diag_d", sqrt(data['sc_h']**2 + data['sc_w']**2)).show()
#data.withColumn("diag_px", sqrt(data['px_height']**2 + data['px_width']**2)).show()
data2 = data.withColumn("PPX", (sqrt(data['px_height']**2 + data['px_width']**2))/(sqrt(data['sc_h']**2 + data['sc_w']**2)/2.54)).sort(desc("PPX"))

## Сохранение


In [79]:
data2.show(20)

+----+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+------------------+
|  id|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|               PPX|
+----+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+------------------+
| 460|         1202|   0|        2.5|       1|  3|     0|        22|  0.4|      112|      2|  8|     1900|    1959| 291|   5|   2|        4|      0|           0|   0|       null|1287.1976878368039|
|null|         1090|   0|        1.3|       1|  5|     0|        27|  0.1|       91|      4| 16|     1638|    1910|1352|   5|   1|       11|      0|           1|   0|          1| 1253.395520950375|
|null|    

In [87]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и общей инфой в формате "2 ядра 2 гига"
# 2 балла
###############
# Ваш код здесь
data2.coalesce(1).write.option("inferSchema","true").option("header","true").option("sep",";").option("encoding","cp1251").mode("overwrite").csv("/content/drive/MyDrive/Colab Notebooks/Spark/data2.csv")

In [88]:
#проверяем сохраненный файл - все на месте
data_data = spark.read.option('header','True').option('inferSchema', 'true').csv('/content/drive/MyDrive/Colab Notebooks/Spark/data2.csv',sep=';')
data_data.show(5)

+----+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+------------------+
|  id|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|               PPX|
+----+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+------------------+
| 460|         1202|   0|        2.5|       1|  3|     0|        22|  0.4|      112|      2|  8|     1900|    1959| 291|   5|   2|        4|      0|           0|   0|       null|1287.1976878368039|
|null|         1090|   0|        1.3|       1|  5|     0|        27|  0.1|       91|      4| 16|     1638|    1910|1352|   5|   1|       11|      0|           1|   0|          1| 1253.395520950375|
|null|    